In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt



## Implementing Vectorized Logistic Regression from Scratch

In this notebook, we will implement a vectorized version of the logistic regression model from scratch. To facilitate this, it's important to understand the dimensions of the variables involved:

### 1. Feature Matrix (`X`)
- **Shape**: `(number_of_features, number_of_examples)`
- **Description**: This matrix contains the input features of the dataset.

### 2. Target Vector (`y`)
- **Shape**: `(1, number_of_examples)`
- **Description**: This vector contains the target values for each training example.

### 3. Weights (`w`)
- **Shape**: `(number_of_features, 1)`
- **Description**: This vector contains the weights associated with each feature.

### 4. Bias (`b`)
- **Shape**: Scalar
- **Description**: The bias term is a scalar that is added to the weighted sum of the input features to adjust the output.


In [2]:
def load_data(zero_rank=False, transformed=False):
    data = np.loadtxt(r'.\data\ex2data1.txt', delimiter=',')

    if data.shape[1] > 2:
        X = data[:, :-1]
        y = data[:, -1]
    else:
        X = data[:, 0]
        y = data[:, 1]
    
    if not zero_rank:
        X = X.reshape(-1, 1) if X.ndim == 1 else X
        y = y.reshape(-1, 1)
    
    if transformed:
        X = X.T
        y = y.T
    
    return X, y


In [3]:
def initialize_parameter_with_zeros(X):
    """
    
    :param X shape: (nx, number_of_features): 
    :return: 
    """
    n, m = X.shape
    
    w = np.zeros((n,1))
    b = 0.0
    return w, b

In [4]:
# initialize_parameter_with_zeros_test
X, y = load_data(transformed=True)
w, b = initialize_parameter_with_zeros(X)
print('w:', w)
print('b:', b)

w: [[0.]
 [0.]]
b: 0.0


In [5]:
def logistic_function(X, w, b):
    """
    :param X: shape (number_of_features, number_of_examples) 
    :param w: shape( number_of_features, 1) 
    :param b: scalar
    :param activation: (linear, sigmoid)
    :return: 
    """

    z = np.dot(w.T, X ) + b
    a = 1/(1 + np.exp(-z))        
    return a

In [6]:
def compute_cost(X, Y, w, b):
    """
    :param X: shape (number_of_features, number_of_examples) 
    :param Y: shape (1, number_of_examples)
    :param w: shape( number_of_features, 1) 
    :param b: scalar
    :return scalar: 
    """
    m = X.shape[1]
    Y_hat = logistic_function(X, w, b)
    
    cost = -np.sum((np.dot(Y, np.log(Y_hat).T) + np.dot(1 - Y, np.log(1 - Y_hat).T))) / m
    return cost

In [7]:
# compute_cost_test

X,y = load_data(transformed=True)

init_w, init_b = initialize_parameter_with_zeros(X)
cost = compute_cost(X, y, init_w, init_b)
print('Cost at initial w and b (zeros): {:.3f}'.format(cost))

Cost at initial w and b (zeros): 0.693


In [8]:
def compute_gradient(X, Y, w, b):
    """
    :param X: shape (number_of_features, number_of_examples) 
    :param Y: shape (1, number_of_examples)
    :param w: shape( number_of_features, 1) 
    :param b: scalar
    :return tuple (number_of_features, 1), scalar: 

    """
    m = X.shape[1]  # number of examples
    
    Y_hat = logistic_function(X, w, b)

    error = Y_hat - Y

    db = np.sum(error) / m
    dw = np.dot(X, error.T) / m
    
    return dw, db

In [9]:
#compute_gradient_test
X, y = load_data(transformed=True)
init_w, init_b = initialize_parameter_with_zeros(X)

dj_dw, dj_db = compute_gradient(X, y, init_w, init_b)
print(f'dj_db at initial w and b (zeros):{dj_db}' )
print(f'dj_dw at initial w and b (zeros):{dj_dw}' )

dj_db at initial w and b (zeros):-0.1
dj_dw at initial w and b (zeros):[[-12.00921659]
 [-11.26284221]]


In [10]:
def gradient_descent(X, Y, w, b, learning_rate=0.01, num_iterations=10000):
    costs = []
        
    for i in range(num_iterations):
        
        dw, db = compute_gradient(X, Y, w, b)
        
        w = w - learning_rate * dw
        b = b - learning_rate * db
        
        cost = compute_cost(X, Y, w, b)
        costs.append(cost)
        
        if i % 1000 == 0:
            print(f"Iteration {i}: Cost = {cost}")
            # Sonuçları yazdır
    print(f"Final Cost at iteration {num_iterations}:{float(costs[-1]):8.2f}")
    return w, b

In [11]:
# gradient_descent_test
X,y = load_data(transformed=True)

np.random.seed(1)
initial_w = (0.01 * (np.random.rand(2) - 0.5)).reshape(-1,1)
initial_b = -8

# Some gradient descent settings
iterations = 10000
learning_rate = 0.001

w,b, = gradient_descent(X ,y, initial_w, initial_b, learning_rate=learning_rate, num_iterations=iterations )

Iteration 0: Cost = 0.9637901832137513
Iteration 1000: Cost = 0.30509032658691554
Iteration 2000: Cost = 0.30472280911605276
Iteration 3000: Cost = 0.30435770911398763
Iteration 4000: Cost = 0.30399500255077383
Iteration 5000: Cost = 0.303634665714201
Iteration 6000: Cost = 0.3032766752046073
Iteration 7000: Cost = 0.3029210079297916
Iteration 8000: Cost = 0.3025676411000218
Iteration 9000: Cost = 0.3022165522231424
Final Cost at iteration 10000:    0.30


In [12]:

def predict(X, w, b):
    """
    :param X: shape (number_of_features, number_of_examples) 
    :param w: shape( number_of_features, 1) 
    :param b: scalar
    """
    
    m = X.shape[1]
    
    y_hat = logistic_function(X, w, b)
    
    predictions = (y_hat > 0.5).astype(int)
    
    return predictions

    

In [16]:
#predict_test

X_train, y_train = load_data(transformed=True)

np.random.seed(1)
init_w, init_b = initialize_parameter_with_zeros(X_train)

# Some gradient descent settings
iterations = 10000
learning_rate = 0.001

w,b, = gradient_descent(X ,y, initial_w, initial_b, learning_rate=learning_rate, num_iterations=iterations )
preds = predict(X_train, w, b)
print(preds)


Iteration 0: Cost = 0.9637901832137513
Iteration 1000: Cost = 0.30509032658691554
Iteration 2000: Cost = 0.30472280911605276
Iteration 3000: Cost = 0.30435770911398763
Iteration 4000: Cost = 0.30399500255077383
Iteration 5000: Cost = 0.303634665714201
Iteration 6000: Cost = 0.3032766752046073
Iteration 7000: Cost = 0.3029210079297916
Iteration 8000: Cost = 0.3025676411000218
Iteration 9000: Cost = 0.3022165522231424
Final Cost at iteration 10000:    0.30
[[0 0 0 1 1 0 1 1 1 1 1 0 1 1 0 1 1 0 1 1 0 1 0 0 1 1 1 1 0 0 1 1 0 1 0 0
  1 1 0 0 1 0 1 1 0 0 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 0 0 0 0 0 1 0 1 1 0 1
  1 1 1 1 1 1 0 1 1 1 1 0 1 1 0 1 1 0 1 1 0 1 1 1 1 1 0 1]]
